This code reads in selection tables created in Raven Pro and prepares the data for analysis.

In [2]:
import pandas as pd
import re
import pyarrow
from datetime import datetime, timedelta
from noaa_coops import Station

Read in Data

In [18]:
# File Path Configuration

# Round 1 20 Hz to 24 kHz
filepaths = [
    "/Volumes/SeaBABELa/BVI Mangroves_2024/BVI Mangroves_2024_Working/6863_Paraquita/6863_Paraquita_Raven/BVI_mangroves_6863_20240115_174358.diversity.selections.txt",
    "/Volumes/SeaBABELa/BVI Mangroves_2024/BVI Mangroves_2024_Working/6879_French/6879_French_Raven/BVI_mangroves_6879_20240115_151620_list.diversity.selections.txt",
    "/Volumes/SeaBABELa/BVI Mangroves_2024/BVI Mangroves_2024_Working/6880_HansA/6880_HansA_Raven/BVI_mangroves_6880_20240116_162417_list.diversity.selections.txt",
    "/Volumes/SeaBABELa/BVI Mangroves_2024/BVI Mangroves_2024_Working/6884_SeaCowBay/6884_SeaCowBay_Raven/BVI_mangroves_6864_20240114_194526.diversity.selections.txt"
]

# --- Initialize List for DataFrames ---
dfs = []

# --- Loop Through Files, Read, and Initial Processing ---
for file in filepaths:
    df = pd.read_csv(file, sep='\t', engine='python')
    df['file'] = file

    # Drop Unwanted Columns
    df = df.drop(columns=['Channel', 'From', 'Tags', 'View'], errors='ignore')

    # Rename Columns
    rename_cols = {
        'Selection': 'selection',
        'Begin Time (s)': 'begin_time_s',
        'End Time (s)': 'end_time_s',
        'Low Freq (Hz)': 'low_freq_hz',
        'High Freq (Hz)': 'high_freq_hz',
        'Begin Path': 'begin_path',
        'File Offset (s)': 'file_offset_s',
        'Peak Freq (Hz)': 'peak_freq_hz',
        'Label': 'label'
    }
    df = df.rename(columns=rename_cols)

    dfs.append(df)

# --- Combine All DataFrames ---
df = pd.concat(dfs, ignore_index=True)

Prepare data

In [5]:
# Extract the hydrophone number (list as site)
df['site'] = df['file'].str.extract(r'BVI_mangroves_(\d{4})_').astype(float)

# Extract Date, Time, and Hour from 'begin_path' Filename
date_time_match = df['begin_path'].str.extract(r'(\d{8})_(\d{6})')

if date_time_match.notna().all().all():
    df['extracted_date_str'] = date_time_match[0]
    df['extracted_time_str'] = date_time_match[1]
    df['extracted_datetime'] = pd.to_datetime(df['extracted_date_str'] + df['extracted_time_str'], format='%Y%m%d%H%M%S')

    # Add 'file_offset_s' as a Timedelta
    df['adjusted_datetime'] = df['extracted_datetime'] + pd.to_timedelta(df['file_offset_s'], unit='s')

    # Update 'date', 'time', and 'hour' from the adjusted datetime
    df['date'] = df['adjusted_datetime'].dt.date
    df['time'] = df['adjusted_datetime'].dt.strftime('%H%M%S')
    df['hour'] = df['adjusted_datetime'].dt.hour.astype(float)

    #Convert date to proper datetime.
    df['date'] = pd.to_datetime(df['date']).dt.tz_localize('UTC')

    # Drop temporary columns
    df = df.drop(columns=['extracted_date_str', 'extracted_time_str', 'extracted_datetime', 'adjusted_datetime'])
else:
    print("Warning: Could not extract date and time information from the filename in 'begin_path' in all rows. Please check the format.")
    df['date'] = pd.NaT
    df['time'] = None
    df['hour'] = None

# --- Create 'tod' Column based on 'hour' ---
def get_tod(hour):
    if hour in [3, 4, 5]:
        return 'midnight'
    elif hour in [9, 10, 11]:
        return 'sunrise'
    elif hour in [15, 16, 17]:
        return 'noon'
    elif hour in [19, 20, 21, 22, 23]:
        return 'sunset'
    return None

df['tod'] = df['hour'].apply(get_tod)

# --- Fetch Tidal Data ---
# NOAA CO-OPS Station Setup
usvi = Station(id="9751419")  # HAULOVER BAY, ST. JOHNS, PR

# Determine the date range from your DataFrame
start_date = df['date'].min()
end_date = df['date'].max()

# Fetch tidal data in UTC
df_tides = usvi.get_data(
    begin_date=start_date.strftime('%Y%m%d'),
    end_date=end_date.strftime('%Y%m%d'),
    product="predictions",
    datum="MLLW", # Mean Lower Low Water provides a conservative estimate of the lowest possible water levels.
    units="metric",
    time_zone="gmt" # important change.
)

# Ensure datetime-aware timestamps
df_tides.index = pd.to_datetime(df_tides.index).tz_localize("UTC")

# Merge tidal height data into your DataFrame
df = pd.merge_asof(
    df.sort_values('date'),
    df_tides[['v']].sort_index(),
    left_on='date',
    right_index=True,
    direction='nearest'
)

# Rename tidal column (not working??)
df_tides.rename(columns={'v': 'mllw'}, inplace=True)

# --- Summarize 'tod' counts by 'date' and 'site' ---
tod_summary = df.groupby(['date', 'site', 'tod']).size().unstack(fill_value=0)

# --- Display the Summary ---
print("\nSummary of Time of Day (tod) counts by Date and Site (Adjusted Time from begin_path filename):")
print(tod_summary)

# --- Display Results ---
print("\nHead of the processed DataFrame (Adjusted Time from begin_path filename):")
print(df.head())
print("\nColumns of the processed DataFrame:")
print(df.columns)

Save data

In [10]:
# Save the main DataFrame to CSV or parquet
df.to_parquet('/Users/jillmunger/Desktop/UNH/research/diversity/div_data_tide.parquet', index=False)

df.to_csv('/Users/jillmunger/Desktop/UNH/research/diversity/div_data_tide.csv', index=False)

In [6]:
import matplotlib.pyplot as plt

# --- Plot just the MLLW tide level over time ---
plt.figure(figsize=(12, 6))
plt.plot(df_tides.index, df_tides['mllw'], label='Tide Height (MLLW)', color='steelblue')
plt.xlabel('Date & Time (UTC)')
plt.ylabel('Tide Height (meters)')
plt.title('NOAA Predicted Tidal Levels (MLLW Datum)')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

NameError: name 'df_tides' is not defined

<Figure size 1200x600 with 0 Axes>